In [1]:
import pandas as pd
data1 = pd.read_excel('../中银微贷款/4月/华诚融通（陕西）-319笔.xls')
data2 = pd.read_excel('../中银微贷款/4月/华诚融通（陕西）-8819笔.xls')
data3 = pd.read_excel('../中银微贷款/4月/华诚融通（陕西）-9374笔.xls')
data4 = pd.read_excel('../中银微贷款/4月/华诚融通（陕西）-10510笔.xls')

In [19]:
import pandas as pd
import numpy as np
import math
import openpyxl
from datetime import *
pd.set_option('display.max_columns',100)
tele_map = pd.read_excel("手机号段分配.xlsx") 
citylevel = pd.read_excel("城市等级.xlsx")
city_map = pd.read_excel("city_map.xlsx")
df = pd.DataFrame()
def clear_char(x):
    if('？' in str(x)):
        x=str(x).replace('？','')
    if(' ' in  str(x)):
        x=str(x).replace(' ','')
    if(':' in str(x)):
         x=str(x).replace(':','')
    return x
def repeat_num(row):
    if(row['联系人手机']==row['手机']):
        return -1;
    else:
        return row['联系人手机']
def repeat_num2(row):
    if(row['第二联系人手机']==row['手机']):
        return -1;
    else:
        return row['第二联系人手机']
def tele_to_int(x):#手机号提取前三位
    if x==-1:
                return x
    else:
        y = np.int64(x)//100000000
        if y>100 and y<200:
            return  y
        else:
            return y%1000
def count_repeat(row):
    count=3
    if(row['手机']==-1):
        count-=1
    if(row['联系人手机']==-1):
        count-=1
    if(row['第二联系人手机']==-1):
        count-=1
    return count
def 单位名称_to_flag(s): 
    if ('有限公司'in str(s)) or ('有限责任公司'in str(s)):
        return 1 
    else: 
        return 0
def sex_to_int(x):
    if str(x) == '男':
        return 0
    else:
        return 1
def age_to_int(x):#年龄分段
    y = str(x).split("-",1)[0]
    y = int(y)
    if y < 1980:
        return 1
    elif (y >= 1995) and (y <= 1997):
        return 0
    elif (y >= 1985) and (y < 1993):
        return 3
    else:  
        return 2
def calc_c(row):
    y = str(row['生日']).split("-",1)[0]
    y = 2017 - int(y)
    if (row['婚姻状况'] is np.nan) and (y>=30):
        return 1
    elif '已婚' in str(row['婚姻状况']):
        return 1
    else:
        return 0
def countryside_c(row):
    x = 2
    #户籍是否为农村
    if '村' in str(row['户籍地址']):
        x=x-1
    #住址是否为农村
    if '村' in str(row['住宅地址']):
        x=x-1
    return x
def nan_to_int(x):
    if x is np.nan:
        return 0
    else:
        return 1
def education_to_int(x):
    if x is np.nan:
        return 1
    elif '本科' in str(x):
        return 3
    elif '专科' in str(x):
        return 2
    elif '职高' in str(x):
        return 1
    elif '高中' in str(x):
        return 0
    else:
        return 1
def relative_to_int(x):
    #if (str(x) == '家人') or (str(x) == '同事'):
    if  x is np.nan:
        return 0
    else:
        return 1
def relative(x):
    if str(x) == '父母':
        return 4
    elif str(x) == '家人':
        return 3
    elif str(x) == '亲戚':
        return 2
    elif str(x) == '朋友':
        return 1
    elif str(x) == '同事':
        return 0
    else:
        return -1
def secondrelative(x):
    if str(x) == '家人':
        return 4
    elif str(x) == '亲戚':
        return 3
    elif str(x) == '同学':
        return 2
    elif str(x) == '朋友':
        return 1
    elif str(x) == '同事':
        return 0
    else:
        return -1
def object_to_int(x):#首逾标识去除最后两个字符
    if  x is np.nan:
        return 0
    else:
        return int(str(x)[:-2])
def object_to_int2(x):#逾期阶段去除第一个字符
    return int(str(x)[1:])
def num_to_cut(x):#身份证取前6位
    return int(str(x)[:6])
def city_to_cut(x):
    return str(x).split("-",2)[1]
def quchong(r_p):
    s = set("")
    count = 0
    for index in r_p.index:
        number = r_p.loc[index, '消费金融账号']
        amount = r_p.loc[index, '交易金额']
        if number in s:
            li = r_p[r_p['消费金融账号']==number].index.tolist()
            old =  r_p.loc[li[0],'交易金额']
            r_p.loc[li[0],'交易金额'] = old+amount
            #print(number+"  : "+str(old)+" -> "+str(old+amount))
            r_p=r_p.drop(index)
            count+=1
        else:
            s.add(number)
    print("重复标签数："+str(count))
    return r_p
def shopaddress(x):
    if ('北京' in str(x)) or ('上海' in str(x)) or ('深圳' in str(x)):
        return 4
    elif ('天津' in str(x)) or ('苏州' in str(x)):
        return 3
    elif ('浙江' in str(x)) or ('广东' in str(x)):
        return 2
    elif ('江苏' in str(x)) or ('福建' in str(x)):
        return 1
    else:
        return 0
#filename = ['16.11','16.12','17.01','17.02','17.03','17.04','17.05','17.06','17.07','17.08','17.09']

def running(file):
    #print('Start processing {}.xlsx ...'.format(file))
    #data = pd.read_excel('../中银微贷款/3月/华诚融通（陕西）-14832笔.xls')
    #data = pd.read_excel('中银8-12月委案及回款/forsql/{}.xls'.format(file))
    ###dataid = pd.read_excel('../中银微贷款/华诚融通（陕西）.xlsx')
    data = file
    print("样本数"+str(len(data)))
    #dataid = pd.read_excel('中银8-12月委案及回款/forsql/{}id.xls'.format(file))#,sheetname=1)
    #检查数据项是否完备
    columns = ['证件号码','生日','性别','婚姻状况','教育程度','住宅地址','手机','单位名称','单位地址','联系人与客户关系','联系人手机','第二联系人与客户关系','第二联系人手机','首逾标识','逾期阶段','逾期天数','逾期次数','累计逾期天数','最大逾期天数','解除逾期所需偿还金额','当前欠款合计','现贷余额','当前剩余应缴利息','当期以前未偿还利息合计','当前剩余应缴滞纳费','当期以前未偿还滞纳费合计','放贷金额','户籍地址',
               '年利率','日利率','受理商户','分期期数','额度代码',
               '首次动用日期','开户日期','首次逾期日期','开始逾期日期',
              '消费金融账号']
    time = ['首次动用日期','开户日期','首次逾期日期','开始逾期日期']
    data = data[columns]
    #处理空格引起的空数据项
    for c in columns:
        data[c] = data[c].apply(lambda x: np.NaN if str(x).isspace() else x)
    data[time] = data[time].astype(datetime)
    #print(data.isnull().sum())
    #处理手机号
    data['手机'] = data['手机'].fillna(-1)
    data['联系人手机'] = data['联系人手机'].fillna(-1)
    data['第二联系人手机'] = data['第二联系人手机'].fillna(-1)
    #清除手机号中非法字符
    data['手机'] = data['手机'].map(clear_char)
    data['联系人手机'] = data['联系人手机'].map(clear_char)
    data['第二联系人手机'] = data['第二联系人手机'].map(clear_char)
    '''    #检查联系人手机和第二联系人手机是否和手机重复
    print(str(len(data[data['联系人手机']==data['手机']]))+'联系人 '+str(len(data[data['第二联系人手机']==data['手机']]))+"第二联系人 与本人手机一致")
    data['联系人手机'] =data.apply(repeat_num, axis=1)
    data['第二联系人手机'] =data.apply(repeat_num2, axis=1)'''
    #提取前三位
    data['手机'] = data['手机'].map(tele_to_int)
    data['联系人手机'] = data['联系人手机'].map(tele_to_int)
    data['第二联系人手机'] = data['第二联系人手机'].map(tele_to_int)
    '''    #号码重复 重复越多值越小 3->2->1->0
    data['号码重复数'] = data.apply(count_repeat,axis=1)
    '''
    #df.rename(columns={'原列名' : '新列名'})#手机号映射为运营商编号
    data = pd.merge(data,tele_map,'left',on='手机')
    data = data.rename(columns={'运营商':'运营商for本人'})
    data = data.drop(['手机'],axis=1) 
    data = data.rename(columns={'联系人手机':'手机'} ) 
    data = pd.merge(data,tele_map,'left',on='手机')
    data = data.rename(columns={'运营商':'运营商for家人'}) 
    data = data.drop(['手机'],axis=1)
    data = data.rename(columns={'第二联系人手机':'手机'}) 
    data = pd.merge(data,tele_map,'left',on='手机') 
    data = data.rename(columns={'运营商':'运营商for同事'}) 
    data = data.drop(['手机'],axis=1)
    #处理单位名称
    data['单位名称'] = data['单位名称'].map(单位名称_to_flag)
    #处理性别
    data['性别'] = data['性别'].map(sex_to_int)
    #处理生日
    data['年龄'] = data['生日'].map(age_to_int)
    #data = data.rename(columns={'生日':'年龄'})
    #判断教育程度和婚姻状况是否为空
    #data['婚姻状况'] = data['生日'].map(marry_to_int)
    data['婚姻状况'] = data.apply(calc_c, axis=1)
    data['教育程度'] = data['教育程度'].map(education_to_int)
    #处理联系人与客户关系和第二联系人与客户关系
    #data['联系人与客户关系'] = data['联系人与客户关系'].map(relative_to_int)
    #data['第二联系人与客户关系'] = data['第二联系人与客户关系'].map(relative_to_int)
    data['联系人与客户关系'] = data['联系人与客户关系'].map(relative)
    data['第二联系人与客户关系'] = data['第二联系人与客户关系'].map(secondrelative)
    #首逾标识去除末尾连个字符
    data['首逾标识'] = data['首逾标识'].map(object_to_int)
    #逾期阶段去除第一个字符
    data['逾期阶段'] = data['逾期阶段'].map(object_to_int2)
    #判断户口和现住地址是否为农村
    data['农村'] = data.apply(countryside_c, axis=1)
    
    #从身份证提取籍贯城市
    data['num'] = data['证件号码'].map(num_to_cut)
    #将身份证前六位映射为对应城市
    data = pd.merge(data,city_map,'left',on='num')
    data = data.drop(['num'],axis=1)
    #城市映射等级
    data = pd.merge(data,citylevel,'left',on='city')
    data = data.rename(columns={'城市等级':'户籍等级'})
    data = data.drop(['city'],axis=1) 
    data = data.rename(columns={'住宅地址':'city'} )
    data['city'] = data['city'].map(city_to_cut)
    data = pd.merge(data,citylevel,'left',on='city')
    data = data.rename(columns={'城市等级':'住宅等级'}) 
    data = data.drop(['city'],axis=1)
    data = data.rename(columns={'单位地址':'city'}) 
    data['city'] = data['city'].map(city_to_cut)
    data = pd.merge(data,citylevel,'left',on='city') 
    data = data.rename(columns={'城市等级':'单位等级'}) 
    data = data.drop(['city'],axis=1)
    data['户籍等级']=data['户籍等级'].fillna(0)
    data['住宅等级']=data['住宅等级'].fillna(0)
    data['单位等级']=data['单位等级'].fillna(0)
    data['运营商for本人']=data['运营商for本人'].fillna(-1)
    data['运营商for家人']=data['运营商for家人'].fillna(-1)
    data['运营商for同事']=data['运营商for同事'].fillna(-1)
    
    #For new data
    data['年利率'] = data['年利率']*1000 #  年利率=日利率*365
    data['日利率'] = data['日利率']*1000
    data['中银微贷款'] = data['额度代码'].map(lambda x: 1 if x=='016313' else 0)
    data['新易贷-微贷款-乐贷款'] = data['额度代码'].map(lambda x: 1 if x=='017313' else 0)
    data['新易贷-微贷款-摩尔龙'] = data['额度代码'].map(lambda x: 1 if x=='019313' else 0)
    data['新易贷-微贷款-一点即用'] = data['额度代码'].map(lambda x: 1 if x=='01A313' else 0)
    data['新易贷-微贷款-友圈贷'] = data['额度代码'].map(lambda x: 1 if x=='01B313' else 0)
    data['新易贷-微贷款-小牛速贷'] = data['额度代码'].map(lambda x: 1 if x=='01C313' else 0)
    data['新易贷-微贷款-京东金融'] = data['额度代码'].map(lambda x: 1 if x=='01G213' else 0)
    data['新易贷-微贷款-微易贷'] = data['额度代码'].map(lambda x: 1 if x=='01H213' else 0)
    data['商户地址'] = data['受理商户'].map(shopaddress)
    data = data.drop(['受理商户','额度代码'], axis = 1)
    z = {0:3, 1:2, 12:1}
    data['分期期数'] = data['分期期数'].map(z)
    '''    data['0期数'] = data['分期期数'].map(lambda x:1 if x==0 else 0)
    data['1期数'] = data['分期期数'].map(lambda x:1 if x==1 else 0)
    data['12期数'] = data['分期期数'].map(lambda x:1 if x==12 else 0)'''
    
    time = ['首次动用日期','开户日期','首次逾期日期','开始逾期日期']
    data['开户动用日期差'] = data.apply(lambda row: (row['首次动用日期']-row['开户日期']).days, axis=1)
    data['开户动用月数差'] = data['开户动用日期差'].map(lambda x:int(x/30))
    data['首逾动用日期差'] = data.apply(lambda row: (row['首次逾期日期']-row['首次动用日期']).days, axis=1)
    data['首逾动用月数差'] = data['首逾动用日期差'].map(lambda x:int(x/30))
    data['最近逾期距首逾日期差'] = data.apply(lambda row: (row['开始逾期日期']-row['首次逾期日期']).days, axis=1)
    data['最近逾期距首逾月数差'] = data['最近逾期距首逾日期差'].map(lambda x:int(x/30))
    data = data.drop(time, axis=1)
    '''
    #标签数据去重
    dataid = dataid.rename(columns={'消费金融账户 ({}月)'.format(file%100):'消费金融账号'})
    dataid['flag'] = 1
    dataid = quchong(dataid)
    print("正样本数:"+str(len(dataid)))
    #打正负样本标签 
    data = pd.merge(data,dataid[['消费金融账号','交易金额','flag']],how='left',on='消费金融账号')
    data['flag'] = data['flag'].fillna(0)
    data['交易金额'] = data['交易金额'].fillna(0)
    '''
    
    #打批次标签
    data['批次'] = 4
    data = data.drop(['户籍地址'], axis =1)
    data = data.drop(['生日'], axis =1)
    print(len(data))
    ###print(len(data[data['flag']==1]))
    '''
    #对所有非01数据归一化
    from sklearn import preprocessing
    columns = ['首逾标识','逾期天数','逾期次数','累计逾期天数','最大逾期天数','解除逾期所需偿还金额','当前欠款合计','现贷余额','当前剩余应缴利息','当期以前未偿还利息合计','当前剩余应缴滞纳费','当期以前未偿还滞纳费合计','放贷金额','运营商for本人','运营商for家人','运营商for同事','户籍等级','住宅等级','单位等级']
    for c in columns:
        #z-score标准化
        data[c] = preprocessing.scale(data[c])
    '''
    #检查是否还有空数据
    t = data.isnull().sum()
    index = t.index#获取列名列表
    for i in index:
        if(t[i]!=0):
            print(i)
    print('check complete!')
    return data

#df=df.drop(['证件号码'],axis=1) 


In [22]:
#data1 = running(data1)
data2 = running(data2)
data3 = running(data3)
data4 = running(data4)

样本数8819


c:\python35\lib\site-packages\ipykernel_launcher.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python35\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
c:\python35\lib\site-packages\ipykernel_launcher.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versu

8819
check complete!
样本数9374
9374
check complete!
样本数10510
10510
check complete!


In [25]:
df.to_excel('微贷款4月无标签.xlsx',index=False)

In [18]:
df = pd.DataFrame()
df=df.append(running(2))

Start processing 2.xlsx ...
样本数14832
14832
check complete!


In [19]:
df.to_excel('微贷款3月12755无标签.xlsx',index=False)

In [17]:
merge

,消费金融账号
52,6010023496154
99,6010024794111
172,6010023937702
173,6010023793100
188,6010022039339
226,6010023832536
227,6010024339258
228,6010023640129
267,6010022775048
317,6010023066244


In [5]:
#用于预测新的一个月的原始数据，包含清洗和转换
headers=pd.read_excel('headers.xlsx')

print(headers.columns.values)
data12 = pd.read_excel('中银8-12月委案及回款\\中银201801数据\\华诚融通（陕西 ）-398笔.xlsx')
data12 = data12[headers.columns.values]

data12.to_excel('处理后数据\\18.01.xlsx',index=False)
print('save complete')
df = running('18.01')

['户籍地址' '证件号码' '生日' '性别' '婚姻状况' '教育程度' '住宅地址' '手机' '单位名称' '单位地址' '联系人与客户关系'
 '联系人手机' '第二联系人与客户关系' '第二联系人手机' '首逾标识' '逾期阶段' '逾期天数' '逾期次数' '累计逾期天数'
 '最大逾期天数' '解除逾期所需偿还金额' '当前欠款合计' '现贷余额' '当前剩余应缴利息' '当期以前未偿还利息合计' '当前剩余应缴滞纳费'
 '当期以前未偿还滞纳费合计' '放贷金额']
save complete
Start processing 18.01.xlsx ...
样本数398
398
check complete!


In [6]:
df = df.drop(['批次','证件号码'],axis=1)
import xgboost as xgb
model=xgb.Booster() #注意：名字要保持一致，否则报错！
model.load_model('CODE\\final.model')
xgb_test=xgb.DMatrix(df)
preds=model.predict(xgb_test)
len(preds)

c:\python35\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


398

In [9]:
pd.DataFrame(preds/2+0.4).describe()

,0
count,398.000000
mean,0.680324
std,0.043148
min,0.579486
25%,0.654361
50%,0.677624
75%,0.705295
max,0.857222


In [10]:
pd.DataFrame(preds/2+0.4).to_excel('preds.xlsx')

Start processing 17.11.xlsx ...
3052
check complete!


In [4]:
m3 = df[df['逾期阶段']==3]
m4 = df[df['逾期阶段']>3]

In [5]:
m4

,证件号码,性别,婚姻状况,教育程度,单位名称,联系人与客户关系,第二联系人与客户关系,首逾标识,逾期阶段,逾期天数,...,运营商for本人,运营商for家人,运营商for同事,年龄,农村,户籍等级,住宅等级,单位等级,flag,批次
0,330122198812200013,0,0,2,0,0,0,6,8,231,...,0,0.0,0.0,3,2,3.0,3.0,3.0,1.0,16.11
1,371202198801174727,1,1,0,1,1,1,15,8,223,...,2,0.0,1.0,3,1,0.0,1.0,1.0,0.0,16.11
2,342225199212210018,0,0,3,1,1,1,2,8,227,...,1,0.0,0.0,3,2,0.0,3.0,3.0,0.0,16.11
3,452123198107015811,0,1,2,1,0,0,9,8,214,...,0,0.0,0.0,2,1,0.0,3.0,3.0,1.0,16.11
4,520102198707245818,0,0,1,1,1,1,12,8,223,...,2,0.0,1.0,3,2,2.0,2.0,2.0,0.0,16.11
5,330302198408032826,1,1,2,0,1,1,11,8,220,...,0,0.0,0.0,2,2,2.0,2.0,2.0,0.0,16.11
6,320902198504297032,0,1,2,1,0,0,11,8,212,...,0,0.0,0.0,3,1,1.0,1.0,1.0,0.0,16.11
7,430421199005098532,0,0,2,0,1,1,3,8,227,...,2,0.0,0.0,3,1,1.0,1.0,1.0,0.0,16.11
8,420581199203110037,0,1,3,1,0,0,10,8,226,...,0,0.0,0.0,3,2,1.0,1.0,1.0,0.0,16.11
9,432501198905016536,0,1,2,1,0,0,2,8,240,...,0,0.0,0.0,3,2,0.0,3.0,3.0,0.0,16.11


In [6]:
m3.to_excel('m3(16.11-17.09).xlsx',index=False)
print('m3 saving done!')
m4.to_excel('m4+(16.11-17.09).xlsx',index=False)
print('m4+ saving done!')

m3 saving done!
m4+ saving done!


In [23]:
#检查同批次是否有同账号
print(len(pd.merge(data1,data2,on='消费金融账号',how='inner')))
print(len(pd.merge(data1,data3,on='消费金融账号',how='inner')))
print(len(pd.merge(data1,data4,on='消费金融账号',how='inner')))
print(len(pd.merge(data2,data3,on='消费金融账号',how='inner')))
print(len(pd.merge(data2,data4,on='消费金融账号',how='inner')))
print(len(pd.merge(data3,data4,on='消费金融账号',how='inner')))

0
0
0
0
0
0


In [24]:
df = data1.append(data2).append(data3).append(data4)
len(df)

29022